<a href="https://colab.research.google.com/github/Samvelus/Credenciamento/blob/main/CERTIFICADOS_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

"""
Gerador automático de certificados a partir de um Excel e um template PDF de 2 páginas.

Requisitos (instalar via pip):
  pip install pandas openpyxl reportlab pypdf2 python-dateutil

Como funciona (resumo):
- Le um arquivo Excel (path perguntado no terminal) com pelo menos duas abas possíveis:
    1) "participantes" — contém a lista de pessoas com colunas esperadas: NOME, TIPO (participante|ministrante), CERT_NUM (opcional)
    2) "cursos" — (opcional) contém curso(s) disponíveis com colunas: CURSO, DATAINICIO, DATAFIM, CARGAHORARIA, PROGRAMA (itens separados por ';'), MINISTRANTES (nomes separados por ';')
  Se a aba "cursos" não existir, o usuário pode digitar manualmente os dados do curso no terminal.

- Pede o template PDF (2 páginas). Na página 1 ele sobrepõe o texto com o texto do certificado.
- Na página 2 escreve o conteúdo programático (coluna esquerda), lista de ministrantes e campos Volume e Número do certificado.
- Pergunta qual curso gerar (mostra lista numerada se houver aba "cursos").
- Pergunta datas, carga horária se não existirem no Excel.
- Pergunta se é para gerar para participantes, ministrantes ou ambos.
- Gera pastas "Participantes" e "Ministrantes" (conforme seleção) e dentro de cada uma cria uma subpasta com data+nome do curso.
- Os certificados são nomeados como "{CERT_NUM if provided else index:04d} - {NOME}.pdf".

Observações:
- O template deve ser um PDF com 2 páginas, tamanho A4 esperado.
- O posicionamento do texto pode ser ajustado nas funções create_overlay_page1 e create_overlay_page2 (coordenadas em pontos do reportlab, origem canto inferior esquerdo).

"""

In [ ]:
!pip install pandas openpyxl reportlab pypdf2 python-dateutil

In [ ]:
import os
import io
import sys
from datetime import datetime
from dateutil import parser as date_parser
import pandas as pd
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import landscape, A4
from reportlab.lib.units import mm
from reportlab.lib.enums import TA_CENTER
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase.pdfmetrics import registerFontFamily
from PyPDF2 import PdfReader, PdfWriter
from PyPDF2._page import PageObject

In [ ]:
# Registrar fontes Arial (Regular, Bold, Italic)
try:
    # Tenta carregar localmente
    pdfmetrics.registerFont(TTFont('Arial', 'Arial.ttf'))
    pdfmetrics.registerFont(TTFont('Arial-Bold', 'Arial Bold.ttf'))
    pdfmetrics.registerFont(TTFont('Arial-Italic', 'Arial Italic.ttf'))

    # VINCULA A FAMÍLIA (Isso faz o <b> funcionar)
    registerFontFamily('Arial', normal='Arial', bold='Arial-Bold', italic='Arial-Italic')
    DEFAULT_FONT = 'Arial'

except Exception:
    try:
        # Tenta carregar do Drive (caminhos absolutos)
        base_font_path = '/content/drive/MyDrive/PET/Certificados/templates/'
        pdfmetrics.registerFont(TTFont('Arial', base_font_path + 'arial.ttf'))
        pdfmetrics.registerFont(TTFont('Arial-Bold', base_font_path + 'arial-bold.ttf'))
        # Se tiver itálico no drive, registre também, senão remova a linha abaixo
        # pdfmetrics.registerFont(TTFont('Arial-Italic', base_font_path + 'arial-italic.ttf'))

        # VINCULA A FAMÍLIA
        registerFontFamily('Arial', normal='Arial', bold='Arial-Bold', italic='Arial-Italic')
        DEFAULT_FONT = 'Arial'
    except Exception as e:
        print(f"Aviso: Não foi possível carregar Arial. Usando Helvetica. Erro: {e}")
        DEFAULT_FONT = 'Helvetica'  # fallback

# Registrar DejaVu como opção secundária
try:
    pdfmetrics.registerFont(TTFont('DejaVuSans', '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf'))
except Exception:
    pass  # fallback
try:
    pdfmetrics.registerFont(TTFont('DejaVuSans', '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf'))
    DEFAULT_FONT = 'DejaVuSans'
except Exception:
    DEFAULT_FONT = 'Helvetica'

PAGE_HEIGHT,PAGE_WIDTH  = A4

def ask(prompt, default=None):
    if default:
        val = input(f"{prompt} [{default}]: ")
        return val.strip() or default
    else:
        return input(f"{prompt}: ").strip()


def read_excel(path):
    xls = pd.ExcelFile(path)
    sheets = {name: xls.parse(name) for name in xls.sheet_names}
    return sheets


def parse_date_br(date_str):
    """
    Converte formatos comuns de data para datetime.
    Aceita:
    - 10/02/2025
    - 10-02-2025
    - 2025-02-10
    - 2025/02/10
    - 10 Fev 2025
    - 10 Fevereiro 2025
    - 10 Feb 2025
    """

    date_str = date_str.strip()

    formatos = [
        "%d/%m/%Y",
        "%d-%m-%Y",
        "%Y-%m-%d",
        "%Y/%m/%d",
        "%d %b %Y",      # 10 Fev 2025
        "%d %B %Y",      # 10 Fevereiro 2025
        "%d %b %y",
        "%d %B %y"
    ]

    for f in formatos:
        try:
            return datetime.strptime(date_str, f)
        except:
            pass

    raise ValueError(
        f"Formato de data inválido: '{date_str}'. "
        "Use formatos como 10/02/2025, 2025-02-10, 10 Fev 2025, etc."
    )


def choose_course(sheets):
    # Espera uma aba 'cursos' com colunas CURSO, DATAINICIO, DATAFIM, CARGAHORARIA, PROGRAMA, MINISTRANTES
    if 'cursos' in sheets:
        df = sheets['cursos']
        # normalizar col names
        df.columns = [c.upper() for c in df.columns]
        if 'CURSO' in df.columns:
            print('\nCursos disponíveis:')
            for i, row in df.iterrows():
                print(f"{i+1}. {row.get('CURSO')}")
            sel = ask('Digite o número do curso (ou ENTER para informar manualmente)', default='')
            if sel:
                try:
                    idx = int(sel)-1
                    row = df.iloc[idx]
                    course = {
                        'CURSO': row.get('CURSO'),
                        'DATAINICIO': str(row.get('DATAINICIO')) if pd.notna(row.get('DATAINICIO')) else '',
                        'DATAFIM': str(row.get('DATAFIM')) if pd.notna(row.get('DATAFIM')) else '',
                        'CARGAHORARIA': str(row.get('CARGAHORARIA')) if 'CARGAHORARIA' in row.index and pd.notna(row.get('CARGAHORARIA')) else '',
                        'PROGRAMA': str(row.get('PROGRAMA')) if 'PROGRAMA' in row.index and pd.notna(row.get('PROGRAMA')) else '',
                        'MINISTRANTES': str(row.get('MINISTRANTES')) if 'MINISTRANTES' in row.index and pd.notna(row.get('MINISTRANTES')) else ''
                    }
                    return course
                except Exception:
                    print('Seleção inválida. Vou pedir manualmente.')
    # manual
    print('\nInforme os dados do curso manualmente:')
    curso = ask('Nome do curso/evento')
    data_inicio = ask('Data de início (YYYY-MM-DD)')
    data_fim = ask('Data de fim (YYYY-MM-DD)')
    carga = ask('Carga horária (horas)')
    programa = ask('Conteúdo programático (separe itens por ; )')
    ministrantes = ask('Ministrantes (separe por ; )')
    return {
        'CURSO': curso,
        'DATAINICIO': data_inicio,
        'DATAFIM': data_fim,
        'CARGAHORARIA': carga,
        'PROGRAMA': programa,
        'MINISTRANTES': ministrantes
    }


def format_date_for_certificate(datestr):
    # tenta parsear várias formas
    try:
        d = date_parser.parse(datestr, dayfirst=False)
        return d.strftime('%d/%m/%Y')
    except Exception:
        return datestr

def create_overlay_page1(text_line1, date_today):
    from reportlab.platypus import Paragraph, Frame
    from reportlab.lib.styles import getSampleStyleSheet

    packet = io.BytesIO()
    c = canvas.Canvas(packet, pagesize=(A4[1], A4[0]))

    styles = getSampleStyleSheet()
    style = styles["Normal"]
    style.alignment = TA_CENTER

    # CORREÇÃO AQUI: Usar a variável global DEFAULT_FONT
    style.fontName = DEFAULT_FONT

    # Se a fonte for Arial, garantimos que o estilo saiba quem é o negrito
    if DEFAULT_FONT == 'Arial':
        style.boldFontName = "Arial-Bold"

    style.allowHTML = True
    style.fontSize = 14
    style.leading = 20

    # Textbox para o parágrafo principal
    frame_width = PAGE_WIDTH * 0.7
    frame_height = 140
    x = (PAGE_WIDTH - frame_width) / 2
    y = PAGE_HEIGHT - 410

    frame = Frame(x, y, frame_width, frame_height, showBoundary=0)

    p = Paragraph(text_line1, style)
    frame.addFromList([p], c)

    # DATA — centralizada
    c.setFont(DEFAULT_FONT, 12)
    c.drawCentredString(PAGE_WIDTH / 2, 60, f"Cuiabá-MT, {date_today}")

    c.save()
    packet.seek(0)
    return packet

def create_overlay_page2(program_items, ministrantes, volume_text, cert_number_text,date_today):
    """Gera overlay para página 2 com duas colunas: esquerda = programa, direita = ministrantes e campos volume/numero."""
    packet = io.BytesIO()
    c = canvas.Canvas(packet, pagesize=(A4[1], A4[0]))
    c.setFont(DEFAULT_FONT, 11)
    margin = 40*mm
    left_x = margin
    right_x = PAGE_WIDTH/2 + 53*mm
    top_y = PAGE_HEIGHT - 45*mm

    # Título do conteúdo programático
    # c.setFont(DEFAULT_FONT, 12)
    # c.drawString(left_x, top_y, 'Conteúdo programático:') # Descrição superior
    # Parte acima permanece igual

    c.setFont(DEFAULT_FONT, 12)
    y = top_y

    from reportlab.platypus import Paragraph, Frame
    from reportlab.lib.styles import getSampleStyleSheet

    styles = getSampleStyleSheet()
    style = styles["Normal"]
    style.fontName = DEFAULT_FONT
    style.fontSize = 11
    style.leading = 14

    # Montar HTML do texto programático
    program_html = ""
    for item in program_items:
        item = item.strip()
        if not item:
            continue

        if item.startswith("-"):
            program_html += f"<br/>• {item[1:].strip()}"
        else:
            program_html += f"<br/><br/>{item}"

    # Criar frame limitado à metade da página
    column_width = PAGE_WIDTH/2 - 130
    frame_program = Frame(left_x, 0, column_width, top_y+45, showBoundary=0)

    p = Paragraph(program_html, style)
    frame_program.addFromList([p], c)


    # Ministrantes
    c.setFont('Arial-Bold', 11)
    c.drawCentredString(right_x, top_y-35, 'Organizadores:')
    c.setFont(DEFAULT_FONT, 8)
    y2 = top_y - 16

    for m in ministrantes:
        c.drawCentredString(right_x, y2-33, m.strip())
        y2 -= 12
        if y2 < 60*mm:
            break

    # Volume e Número do Certificado (posicionados na parte inferior direita)
    c.setFont(DEFAULT_FONT, 12)
    c.drawCentredString(right_x+57, 77*mm, f'{volume_text}')
    c.drawCentredString(right_x+57, 67*mm, f'{cert_number_text}')

    # DATA — centralizada
    c.setFont(DEFAULT_FONT, 12)
    c.drawCentredString(PAGE_WIDTH / 2, 60, f"Cuiabá-MT, {date_today}")


    c.save()
    packet.seek(0)
    return packet

def split_text_to_lines(text, max_width=200):
    words = text.split()
    lines = []
    cur = ''
    for w in words:
        if len(cur) + len(w) + 1 <= max_width:
            cur = (cur + ' ' + w).strip()
        else:
            lines.append(cur)
            cur = w
    if cur:
        lines.append(cur)
    return lines

def merge_overlay_with_template(template_reader, overlay_packet, page_number, writer):
    overlay_pdf = PdfReader(overlay_packet)
    overlay_page = overlay_pdf.pages[0]

    # Página base do template
    template_page = template_reader.pages[page_number]

    # Cria página vazia do tamanho exato do template
    new_page = PageObject.create_blank_page(
        None,
        template_page.mediabox.width,
        template_page.mediabox.height
    )

    # Primeiro copia o template original
    new_page.merge_page(template_page)

    # Agora coloca o overlay por cima
    new_page.merge_page(overlay_page)

    # Adiciona ao writer
    writer.add_page(new_page)

def ensure_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
   def main():
    print('=== Gerador de Certificados ===')
    excel_path = '/content/drive/MyDrive/PET/Certificados/Planilhas/modelo_certificados.xlsx'
    if not os.path.exists(excel_path):
        print('Arquivo não encontrado. Saindo.')
        sys.exit(1)
    sheets = read_excel(excel_path)

    participantes_df = None
    if 'participantes' in sheets:
        participantes_df = sheets['participantes']
        participantes_df.columns = [c.upper() for c in participantes_df.columns]
    else:
        # tentar pegar primeira sheet como lista de participantes
        first = list(sheets.keys())[0]
        participantes_df = sheets[first]
        participantes_df.columns = [c.upper() for c in participantes_df.columns]

    # Escolher curso
    course = choose_course(sheets)
    curso_nome = course['CURSO']
    data_inicio = format_date_for_certificate(course['DATAINICIO'])
    data_fim = format_date_for_certificate(course['DATAFIM'])
    carga = course['CARGAHORARIA']
    programa_raw = course['PROGRAMA']
    ministrantes_raw = course['MINISTRANTES']

    # Processar programa e ministrantes
    program_items = [p.strip() for p in str(programa_raw).split(';') if p.strip()]
    ministrantes = [m.strip() for m in str(ministrantes_raw).split(';') if m.strip()]

    # Template
    template_path = '/content/drive/MyDrive/PET/Certificados/templates/templade_V1.pdf'
    if not os.path.exists(template_path):
        print('Template PDF não encontrado. Saindo.')
        sys.exit(1)

    # Perguntar se será para participantes, ministrantes ou ambos
    tipo_geracao = ask('Gerar certificados para (participantes / ministrantes / ambos)', default='participantes').lower()
    tipos = []
    if tipo_geracao in ['participantes', 'ambos']:
        tipos.append('participante')
    if tipo_geracao in ['ministrantes', 'ambos']:
        tipos.append('ministrante')

    # Data de hoje
    data_hoje = ask('Data que aparecerá no certificado (ENTER para hoje)', default=datetime.now().strftime('%d/%m/%Y'))

    # Pasta base de saída
    base_out = '/content/drive/MyDrive/PET/Certificados'
    ensure_folder(base_out)

    # Pasta específica por tipo
    for t in tipos:
        ensure_folder(os.path.join(base_out, t.capitalize() + 's'))

    # Ler template
    template_reader = PdfReader(template_path)
    if len(template_reader.pages) < 2:
        print('O template deve ter ao menos 2 páginas. Saindo.')
        sys.exit(1)

    # Cert number start
    cert_start = ask('Número inicial do certificado (ENTER para 1)', default='1')
    try:
        cert_counter = int(cert_start)
    except:
        cert_counter = 1

    # Volume default
    volume_text = ask('Volume (coloque vazio se não usar)', default='')

    # Filtrar participantes por tipo
    # Espera que na tabela exista coluna TIPO (ou se faltar, gera para todos conforme seleção acima)
    df = participantes_df.copy()

    if 'TIPO' not in df.columns:
        # perguntar quem gerar
        print('A planilha de participantes não possui coluna "TIPO". Assumirei que todas as linhas são participantes.')
        df['TIPO'] = 'participante'
    else:
        # normalizar strings
        df['TIPO'] = df['TIPO'].astype(str).str.lower()

    # Filtrar conforme tipos escolhidos
    # Criar lista final de pessoas para gerar certificados
    df_to_generate = pd.DataFrame()

    # Participantes vêm da aba "participantes"
    if 'participante' in tipos:
        if 'TIPO' not in participantes_df.columns:
            participantes_df['TIPO'] = 'participante'
        participantes_df['TIPO'] = participantes_df['TIPO'].astype(str).str.lower()
        part_df = participantes_df[participantes_df['TIPO'] == 'participante']
        df_to_generate = pd.concat([df_to_generate, part_df], ignore_index=True)

    # Ministrantes vêm EXCLUSIVAMENTE da aba "cursos"
    if 'ministrante' in tipos:
        minist_df = pd.DataFrame({
            'NOME': ministrantes,
            'TIPO': ['ministrante'] * len(ministrantes)
        })
        df_to_generate = pd.concat([df_to_generate, minist_df], ignore_index=True)

    if df_to_generate.empty:
        print('Nenhuma pessoa encontrada para gerar certificados.')
        sys.exit(1)


    # Criar subpasta com data e nome do curso
    dt_hoje = parse_date_br(data_hoje)
    date_folder_name = dt_hoje.strftime('%d-%m-%Y') + ' - ' + ''.join([c for c in curso_nome if c.isalnum() or c.isspace()]).strip()

    # Iterar e gerar
    total = len(df_to_generate)
    print(f'Gerando {total} certificados...')
    for idx, row in df_to_generate.iterrows():
        nome = row.get('NOME') if 'NOME' in row.index else row.get('Nome')
        if pd.isna(nome):
            continue
        tipo = row.get('TIPO')
        tipo_folder = 'Participantes' if tipo.startswith('part') else 'Ministrantes'
        out_folder = os.path.join(base_out, tipo_folder, date_folder_name)
        ensure_folder(out_folder)

        # número do certificado — se a planilha fornece, usa, senão usa contador
        cert_num = None
        if 'CERT_NUM' in row.index and pd.notna(row['CERT_NUM']):
            cert_num = str(row['CERT_NUM'])
        else:
            cert_num = f"{cert_counter:04d}"
            cert_counter += 1

        # Valores padrão vindos do curso
        carga_final = carga
        programa_final = program_items

        # Se participante tiver carga horária individual
        if 'CARGAHORARIA' in row.index and pd.notna(row['CARGAHORARIA']):
            carga_final = str(int(row['CARGAHORARIA']))

        # Se participante tiver conteúdo programático individual
        if 'PROGRAMA' in row.index and pd.notna(row['PROGRAMA']):
            programa_final = [p.strip() for p in str(row['PROGRAMA']).split(';') if p.strip()]

        # montar texto da página 1
        papel = 'organizou o' if tipo.startswith('min') else 'participou do'
        # Determina se o período é 1 dia só ou vários
        if data_inicio == data_fim:
            periodo_texto = f"realizado no dia {data_inicio},"
        else:
            periodo_texto = f"realizado de {data_inicio} a {data_fim},"

        # Monta o texto com tags HTML para negrito
        text_line1 = (
            f"Certificamos que <b>{nome}</b> {papel} <b>{curso_nome}</b>, "
            f"{periodo_texto} com carga horária de {carga_final} horas."
        )

        # criar overlays
        overlay1 = create_overlay_page1(text_line1, data_hoje)
        overlay2 = create_overlay_page2(program_items, ministrantes, volume_text, cert_num,data_hoje)

        # merge
        writer = PdfWriter()
        # copy template pages before merging to avoid modifying original reader
        # página 0
        merge_overlay_with_template(template_reader, overlay1, 0, writer)
        # página 1
        merge_overlay_with_template(template_reader, overlay2, 1, writer)

        out_name = f"{cert_num} - {''.join([c for c in nome if c.isalnum() or c.isspace()]).strip()}.pdf"
        out_path = os.path.join(out_folder, out_name)
        with open(out_path, 'wb') as f:
            writer.write(f)

        print(f'Gerado: {out_path}')

    print('Concluído.')


if __name__ == '__main__':
    main()

=== Gerador de Certificados ===

Cursos disponíveis:
1. Introdução à Eletricidade
2. Oficina de Instrumentos de Bancadas
3. PIC (PROGRAMA DE INTEGRAÇÃO DE CALOUROS)
4. Minicurso de Introdução à Eletrônica Básica
5. Curso de Arduino
Digite o número do curso (ou ENTER para informar manualmente): 5
Gerar certificados para (participantes / ministrantes / ambos) [participantes]: ministrantes
Data que aparecerá no certificado (ENTER para hoje) [18/12/2025]: 
Número inicial do certificado (ENTER para 1) [1]: 2171
Volume (coloque vazio se não usar): 1
Gerando 5 certificados...
Gerado: /content/drive/MyDrive/PET/Certificados/Ministrantes/18-12-2025 - Curso de Arduino/2171 - Bruno Vinícius da Silva Ojeda.pdf
Gerado: /content/drive/MyDrive/PET/Certificados/Ministrantes/18-12-2025 - Curso de Arduino/2172 - Jeane Felicio Lisboa.pdf
Gerado: /content/drive/MyDrive/PET/Certificados/Ministrantes/18-12-2025 - Curso de Arduino/2173 - Bianca Rodrigues de Souza.pdf
Gerado: /content/drive/MyDrive/PET/Certif

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
